In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# Scrape data

This notebook scrapes data from the Year End World Top 10 sections of the [Men's](https://en.wikipedia.org/wiki/Official_Men%27s_Squash_World_Ranking) and [Women's](https://en.wikipedia.org/wiki/Official_Women%27s_Squash_World_Ranking) Squash World Ranking wikipedia pages and stores the data in `csv` files. The only way to fully understand how the functions in this notebook work is to visit the pages yourself and inspect the HTML.

Some things to notice:

* the first column of the tables are just the ranks 1 to 10. hence we ignore them. this is done by using `[1:]` when looping through columns in `table_to_pandas`
* in some tables they include the ranking points as well as the player names. we ignore ranking points. this is done by creating function `is_not_numeric`
* the top row of each table is the years and the other rows contains player information. hence in `table_to_pandas` we have two separate list comprehensions, one for the years and one for the player names
* the first table of the section in the mens page has lots of blank entries and does not fit the pattern of the other tables. i have chosen to just ignore the first table. see `url_to_pandas`

In [ ]:
urls = ['https://en.wikipedia.org/wiki/Official_Women%27s_Squash_World_Ranking',
    'https://en.wikipedia.org/wiki/Official_Men%27s_Squash_World_Ranking']

In [ ]:
def _is_not_numeric(s):
    """
    check if the string s is numeric
    
    Args:
        s, string
    Returns:
        boolean
    """
    try:
        float(s)
    except ValueError:
        return True
    else:
        return False


def _table_to_pandas(table):
    """
    extract information from a single table from the Year End World Top 10
    section of the squash world ranking wiki pages, and store it in a pandas
    frame
    
    Args:
        table
            a string of an html table from the squash wiki page
    
    Returns:
        pd.dataframe
            index is from 1 to 10
            column names are the years
            entries are player names
    """
    rows = table.find_all("tr")
    
    # the top row (hence rows[0]) of each table contains the years
    # first column of each table contains ranks, hence why we have [1:]
    years = [col.text.replace("\n", "") for col in rows[0].find_all("th")[1:]]

    # the other rows (hence rows[1:]) of each table contains the players' names
    players = [
        [
            col.text.replace("\n", "")
            for col in row.find_all("td")[1:]
            if _is_not_numeric(col.text.replace("\n", ""))
        ]
        for row in rows[1:]
    ]

    return pd.DataFrame(players, columns=years, index=range(1, 11))


def url_to_pandas(url, ignore_first_table = False):
    """
    extract information from all tables from the Year End World Top 10
    section of the squash world ranking wiki pages, and store it in a pandas
    frame
    
    Args:
        url
            a string containing url to the wiki page
    
    Returns:
        df_stack
            a pd.DataFrame with three columns: rank, year and player
    """
    html = requests.get(url).text
    
    # start and end determined by manually inspecting html of the wiki pages
    start = html.find('id="Year_end_world_top_10_players')
    end = html.find('id="Year-end_number_1')
    
    tables = BeautifulSoup(html[start:end], "html.parser").find_all("table")
    
    # in mens page, first table in the wiki page does not fit the pattern
    if ignore_first_table:
        tables = tables[1:]
    
    df = pd.concat([_table_to_pandas(t) for t in tables], axis=1)
    
    # in general, df.stack() creates new frame whose multiindex consists of the
    # the index and columns of df
    # in this instance, df.stack's multiindexx is [rank, year] and its single
    # column contains names of players
    df_stack = df.stack().reset_index()
    df_stack.columns = ["rank", "year", "player"]

    return df_stack

In [ ]:
df_m = url_to_pandas(urls[1], ignore_first_table=True)
df_f = url_to_pandas(urls[0])

In [ ]:
df_m.to_csv('male_raw.csv')
df_f.to_csv('female_raw.csv')

# Process data

In [ ]:
def player_summaries(df):
    players = df.groupby("player").agg(
        {"rank": [np.mean, "count", np.min, np.max], "year": [np.min, np.max]}
    )

    players.columns = [
        "average_rank",
        "years_in_top10",
        "best_rank",
        "worst_rank",
        "earliest_year",
        "latest_year",
    ]
    players.sort_values(by=["average_rank"], inplace=True)

    return players

In [ ]:
players_f = player_summaries(pd.read_csv('female_raw.csv', index_col=0))
players_m = player_summaries(pd.read_csv('male_raw.csv', index_col=0))

In [ ]:
players_m.to_csv('male.csv')
players_f.to_csv('female.csv')

# visuals and clustering and dimensionality reduction

In [ ]:
players_m = pd.read_csv('male.csv', index_col=0)

In [ ]:
players_m